In [19]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(42)

In [20]:
data1 = pd.read_csv('seeds.csv')
label1 = data1.iloc[:, -1]
data1 = data1.iloc[:, :-1]
for col in data1.columns:
    data1[col] = (data1[col] - data1[col].mean()) / data1[col].std()
datas = data1.values
labels = label1.values

In [21]:
data2 = pd.read_csv('Vowel.csv')
label2 = data2.iloc[:, -1]
data2 = data2.iloc[:, :-1]
for col in data2.columns:
    data2[col] = (data2[col] - data2[col].mean()) / data2[col].std()
dataV = data2.values

def replace_had_with_1(label2):
    mask = (label2 == 'had')
    label2[mask] = 0
    mask = (label2 == 'hAd')
    label2[mask] = 1
    mask = (label2 == 'hed')
    label2[mask] = 2
    mask = (label2 == 'hEd')
    label2[mask] = 3
    mask = (label2 == 'hid')
    label2[mask] = 4
    mask = (label2 == 'hId')
    label2[mask] = 5
    mask = (label2 == 'hod')
    label2[mask] = 6
    mask = (label2 == 'hOd')
    label2[mask] = 7
    mask = (label2 == 'hud')
    label2[mask] = 8
    mask = (label2 == 'hUd')
    label2[mask] = 9
    mask = (label2 == 'hYd')
    label2[mask] = 10

replace_had_with_1(label2)
labelV = label2.values

In [22]:
def k_means(initial, k, stopping=100, error=0.0001):
    n = initial.shape[0]
    d = initial.shape[1]
    centers = initial[random.sample([i for i in range(initial.shape[0])],k)]
    for iteration in range(stopping):
        distances = np.linalg.norm(centers[:, np.newaxis] - centers, axis=2)
        d_dict = {}
        for i in range(k):
            for j in range(k):
                if i != j:
                    d_dict[(i, j)] = distances[i, j]
            d_dict[(i, i)] = 0
        labels = np.zeros(n, dtype=int)
        for i in range(n):
            x = initial[i]
            condition = False
            if (labels[i], labels[i]) in d_dict:
                for j in range(k):
                    if j != labels[i] and 2 * np.linalg.norm(centers[j] - x) <= d_dict[(labels[i], j)]:
                        labels[i] = j
                        condition = True
                        break
            else:
                for j in range(k):
                    if 2 * np.linalg.norm(centers[j] - x) <= d_dict[(labels[i], j)]:
                        labels[i] = j
                        condition = True
                        break
            if not condition:
                for j in range(k):
                    if 2 * np.linalg.norm(centers[j] - x) <= d_dict[(labels[i], j)]:
                        labels[i] = j
                        break
        new_centers = np.zeros((k, d))
        counts = np.zeros(k)
        for i in range(n):
            label = labels[i]
            new_centers[label] += initial[i]
            counts[label] += 1
        for j in range(k):
            if counts[j] > 0:
                new_centers[j] /= counts[j]
        if np.linalg.norm(new_centers - centers) < error:
            break
        centers = new_centers
    return centers

In [23]:
def calculate_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

def silhouette_coefficient(data, labels):
    n = data.shape[0]
    silhouette_scores = np.zeros(n)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distances[i, j] = calculate_distance(data[i], data[j])
            distances[j, i] = distances[i, j]
    for i in range(n):
        label = labels[i]
        label_indices = np.where(labels == label)[0]
        a_i = np.mean(distances[i, label_indices])
        b_distances = []
        for labelb in np.unique(labels):
            if labelb != label:
                labelb_indices = np.where(labels == labelb)[0]
                b_i = np.mean(distances[i, labelb_indices])
                b_distances.append(b_i)
        if len(b_distances)>0:
            b_i = min(b_distances)
            silhouette_scores[i] = (b_i - a_i) / max(a_i, b_i)
    return np.mean(silhouette_scores)

In [24]:
def rand_index(labels_true, labels_pred):
    n = labels_true.shape[0]
    r = 0
    s = 0
    for i in range(n):
        for j in range(i+1, n):
            if labels_true[i] == labels_true[j] and labels_pred[i] == labels_pred[j]:
                r += 1
            elif labels_true[i] != labels_true[j] and labels_pred[i] != labels_pred[j]:
                s += 1
    rand_index = (r + s) / (n * (n - 1) / 2)
    return rand_index

In [25]:
def nmi(labels_true, labels_pred):
    n = labels_true.shape[0]
    clusters = []
    for label in labels_pred:
        if label not in clusters:
            clusters.append(label)
    n_clusters = len(clusters)
    classes = []
    for label in labels_true:
        if label not in classes:
            classes.append(label)
    n_classes = len(classes)
    joint = np.zeros((n_clusters, n_classes))
    labels_pred = labels_pred.astype(int)
    labels_true = labels_true.astype(int)
    for i in range(n):
        joint[clusters.index(labels_pred[i]), classes.index(labels_true[i])] += 1
    mutual = 0
    for i in range(n_clusters):
        for j in range(n_classes):
            pro_xy = joint[i, j] / n
            pro_x = np.sum(joint[i, :]) / n
            pro_y = np.sum(joint[:, j]) / n
            if pro_xy > 0:
                mutual += pro_xy * np.log(pro_xy / (pro_x * pro_y))
            elif pro_x == 0 and pro_y == 0:
                mutual += 0
    h_x = 0
    for i in range(n_clusters):
        pro_x = np.sum(joint[i, :]) / n
        if pro_x > 0:
            h_x += -pro_x * np.log(pro_x)
    h_y = 0
    for j in range(n_classes):
        pro_y = np.sum(joint[:, j]) / n
        if pro_y > 0:
            h_y += -pro_y * np.log(pro_y)
    nmi = mutual / np.sqrt(h_x * h_y)
    return nmi

In [26]:
k_values = [1,2,3]
for k in k_values:
    k_centers = k_means(datas, k)
    k_labels = []
    for data in datas:
        distances = []
        for center in k_centers:
            distance3 = np.linalg.norm(data - center)
            distances.append(distance3)
        cluster_label = np.argmin(distances)
        k_labels.append(cluster_label)
    k_labels = np.array(k_labels)
    if k != 1:
        k_means_silhouette = silhouette_coefficient(datas, k_labels)
    k_means_rand_index = rand_index(labels, k_labels)
    k_means_nmi = nmi(labels, k_labels)
    if k != 1:
        print("seeds: K-means Silhouette Coefficient ( k =", k,"):", k_means_silhouette)
        print()
    
    print("seeds: K-means Rand Index ( k =", k,"):", k_means_rand_index)
    print()
    print("seeds: K-means NMI ( k =", k,"):", k_means_nmi)
    if k != 3:
        print("--------------------------------------------------")

C:\Users\User\AppData\Local\Temp\ipykernel_29252\1568060251.py:38: RuntimeWarning: invalid value encountered in scalar divide
  nmi = mutual / np.sqrt(h_x * h_y)


seeds: K-means Rand Index ( k = 1 ): 0.33014354066985646

seeds: K-means NMI ( k = 1 ): nan
--------------------------------------------------
seeds: K-means Silhouette Coefficient ( k = 2 ): 0.18988323635968601

seeds: K-means Rand Index ( k = 2 ): 0.6126224652540442

seeds: K-means NMI ( k = 2 ): 0.2653519016497088
--------------------------------------------------
seeds: K-means Silhouette Coefficient ( k = 3 ): 0.393489733139898

seeds: K-means Rand Index ( k = 3 ): 0.8808384597858282

seeds: K-means NMI ( k = 3 ): 0.7001411231554484


In [27]:
k_values = [1,2,3]
for k in k_values:
    k_centers = k_means(dataV, k)
    k_labels = []
    for data in dataV:
        distances = []
        for center in k_centers:
            distance4 = np.linalg.norm(data - center)
            distances.append(distance4)
        cluster_label = np.argmin(distances)
        k_labels.append(cluster_label)
    k_labels = np.array(k_labels)
    if k != 1:
        k_means_silhouette = silhouette_coefficient(dataV, k_labels)
    k_means_rand_index = rand_index(labelV, k_labels)
    k_means_nmi = nmi(labelV, k_labels)
    if k != 1:
        print("Vowel: K-means Silhouette Coefficient ( k =", k, "):", k_means_silhouette)
        print()
    
    print("Vowel: K-means Rand Index ( k =", k, "):", k_means_rand_index)
    print()
    print("Vowel: K-means NMI ( k =", k, "):", k_means_nmi)
    if k != 3:
        print("--------------------------------------------------")

Vowel: K-means Rand Index ( k = 1 ): 0.08998988877654196

Vowel: K-means NMI ( k = 1 ): nan
--------------------------------------------------


C:\Users\User\AppData\Local\Temp\ipykernel_29252\1568060251.py:38: RuntimeWarning: invalid value encountered in scalar divide
  nmi = mutual / np.sqrt(h_x * h_y)


Vowel: K-means Silhouette Coefficient ( k = 2 ): 0.20336546131615024

Vowel: K-means Rand Index ( k = 2 ): 0.2747597307759087

Vowel: K-means NMI ( k = 2 ): 0.12333543871810272
--------------------------------------------------
Vowel: K-means Silhouette Coefficient ( k = 3 ): 0.07599381888131675

Vowel: K-means Rand Index ( k = 3 ): 0.3783599391283921

Vowel: K-means NMI ( k = 3 ): 0.08200758580502908
